In [ ]:
import json
import requests


esri_url = "https://services2.arcgis.com/jUpNdisbWqRpMo35/arcgis/rest/services/GEM_ew_21/FeatureServer/0/query?where=1%3D1&objectIds=6303&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&returnCentroid=false&featureEncoding=esriCompressedShapeBuffer&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=4326&defaultSR=4326&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

geojson_data = json.loads(requests.get(esri_url).text)["features"][0]["geometry"]

In [3]:
from typing import Dict, List, Union, Any
import logging

def create_tomtom_polygon(geometry_data: Dict[str, Any]) -> Union[Dict[str, Any], str]:
    """
    Creates a payload for TomTom's Geometry Search API from a given geometry data.

    Parameters:
    geometry_data : dict
        The geometry data containing the type and coordinates of the polygon.
        Example:
        {
            'type': 'Polygon',
            'coordinates': [[[lon1, lat1], [lon2, lat2], ...]]
        }

    Returns:
    dict or str
        A dictionary containing the payload for TomTom's Geometry Search API, or
        a string message if the input geometry type is not 'Polygon'.

    Raises:
    ValueError : If the geometry type is not 'Polygon'.
    KeyError : If 'type' or 'coordinates' keys are missing in the input dictionary.
    """
    # Validate the existence of required keys
    if 'type' not in geometry_data or 'coordinates' not in geometry_data:
        raise KeyError("Missing required keys 'type' or 'coordinates' in the geometry data.")

    # Validate the geometry type
    if geometry_data['type'] != 'Polygon':
        raise ValueError("Invalid geometry type. Only 'Polygon' is supported.")

    # Extract the coordinates
    coordinates = geometry_data.get('coordinates', [[]])[0]
    
    # Validate the coordinates
    if not all(isinstance(coord, (list, tuple)) and len(coord) == 2 for coord in coordinates):
        raise ValueError("Invalid coordinates format. Must be a list of [longitude, latitude].")

    # Convert coordinates to the required "latitude,longitude" string format
    formatted_vertices = [f"{lat},{lon}" for lon, lat in coordinates]
    
    # Create the payload
    polygon = {
        "geometryList": [
            {
                "type": "POLYGON",
                "vertices": formatted_vertices
            }
        ]
    }

    # Validate the polygon structure
    if "geometryList" not in polygon:
        logging.error("Missing 'geometryList' key in polygon")
        return {"error": "Missing 'geometryList' key in polygon"}
        
    for geometry in polygon["geometryList"]:
        if "type" not in geometry or "vertices" not in geometry:
            logging.error("Each geometry in 'geometryList' must contain 'type' and 'vertices' keys")
            return {"error": "Each geometry in 'geometryList' must contain 'type' and 'vertices' keys"}
        
        num_vertices = len(geometry.get("vertices", []))
        
        if num_vertices < 3 or num_vertices > 50:
            logging.error(f"The number of vertices should be between 3 and 50. Found {num_vertices} vertices.")
            return {"error": f"The number of vertices should be between 3 and 50. Found {num_vertices} vertices."}
    
    return polygon

geojson_data = {
        "type" : "Polygon", 
        "coordinates" : 
        [
          [
            [11.5463712494602, 48.228361012576], 
            [11.7147558500941, 48.13967240047], 
            [11.548902827695, 48.071335704101], 
            [11.3739185049929, 48.1558430293687], 
            [11.5463712494602, 48.228361012576]
          ]
        ]
      }


payload = create_tomtom_polygon(geojson_data)


{'geometryList': [{'type': 'POLYGON',
   'vertices': ['48.228361012576,11.5463712494602',
    '48.13967240047,11.7147558500941',
    '48.071335704101,11.548902827695',
    '48.1558430293687,11.3739185049929',
    '48.228361012576,11.5463712494602']}]}

In [4]:
import json

json.dumps(payload)

'{"geometryList": [{"type": "POLYGON", "vertices": ["48.228361012576,11.5463712494602", "48.13967240047,11.7147558500941", "48.071335704101,11.548902827695", "48.1558430293687,11.3739185049929", "48.228361012576,11.5463712494602"]}]}'

In [ ]:
from typing import Union, Optional, List, Dict
from urllib.parse import quote, unquote
import logging
import json
import requests

# Configure logging
logging.basicConfig(level=logging.INFO)

def ensure_url_encoded(query_string: str) -> str:
    """
    Ensure that the query_string is URL-encoded.

    Args:
        query_string (str): The query string that may or may not be URL-encoded.

    Returns:
        str: The URL-encoded version of the query string.
    """
    # First, try to URL-decode the string. If decoding changes the string, then it was URL-encoded.
    try:
        decoded = unquote(query_string)
        if decoded == query_string:
            logging.info("URL encoding the query string.")
            return quote(query_string)
        else:
            logging.info("The query string is already URL-encoded.")
            return query_string
    except Exception as e:
        logging.error(f"An error occurred while ensuring URL encoding: {e}")
        return None

def get_pois_by_polygon(
        api_key: str,
        polygon: Dict,
        query_string: str,
        versionNumber: int = 2,
        ext: str = "json",
        limit: Optional[int] = 100, 
        language: Optional[str] = None, 
        extendedPostalCodesFor: Optional[str] = None, 
        idxSet: Optional[str] = None,
        categorySet: Optional[str] = None, 
        brandSet: Optional[str] = None, 
        connectorSet: Optional[str] = None, 
        minPowerKW: Optional[float] = None, 
        maxPowerKW: Optional[float] = None, 
        fuelSet: Optional[List[str]] = None, 
        view: Optional[str] = None, 
        openingHours: Optional[str] = None, 
        timeZone: Optional[str] = None, 
        mapcodes: Optional[str] = None, 
        relatedPois: Optional[str] = None,
        entityTypeSet: Optional[str] = None) -> Union[Dict, Dict[str, str]]:
    """
    Get Points of Interest (POIs) based on the given parameters.
    
    Args:
        polygon (Dict): The polygon to search within.
        api_key (str): The API key for TomTom.
        query_string (str): The query string (can be URL encoded or will be decoded).
        versionNumber (int, optional): The API version number. Defaults to 2.
        ext (str, optional): The response format. Defaults to 'json'.
        limit (Optional[int], optional): Maximum number of search results [1, 100]. Defaults to 100.
        language (Optional[str], optional): Language for search results. Defaults to None.
        extendedPostalCodesFor (Optional[str], optional): Extended postal codes. Defaults to None.
        idxSet (Optional[str], optional): Indexes for search. Defaults to None.
        categorySet (Optional[str], optional): Categories to restrict results. Defaults to None.
        brandSet (Optional[str], optional): Brands to restrict results. Defaults to None.
        connectorSet (Optional[str], optional): Connector types for EV stations. Defaults to None.
        minPowerKW (Optional[float], optional): Minimum power in KW for EV stations. Defaults to None.
        maxPowerKW (Optional[float], optional): Maximum power in KW for EV stations. Defaults to None.
        fuelSet (Optional[List[str]], optional): Fuel types to restrict results. Defaults to None.
        view (Optional[str], optional): Geopolitical view. Defaults to None.
        openingHours (Optional[str], optional): Opening hours for POI. Defaults to None.
        timeZone (Optional[str], optional): Time zone mode. Defaults to None.
        mapcodes (Optional[str], optional): Mapcodes. Defaults to None.
        relatedPois (Optional[str], optional): Related POIs. Defaults to None.
        entityTypeSet (Optional[str], optional): Entity types for geography results. Defaults to None.

    Returns:
        Union[Dict, Dict[str, str]]: Either the Points of Interest (POIs) or an error message.
    """
    
    # Validate the polygon structure
    if "geometryList" not in polygon:
        logging.error("Missing 'geometryList' key in polygon")
        return {"error": "Missing 'geometryList' key in polygon"}
        
    for geometry in polygon["geometryList"]:
        if "type" not in geometry or "vertices" not in geometry:
            logging.error("Each geometry in 'geometryList' must contain 'type' and 'vertices' keys")
            return {"error": "Each geometry in 'geometryList' must contain 'type' and 'vertices' keys"}
        
        num_vertices = len(geometry.get("vertices", []))
        
        if num_vertices < 3 or num_vertices > 50:
            logging.error(f"The number of vertices should be between 3 and 50. Found {num_vertices} vertices.")
            return {"error": f"The number of vertices should be between 3 and 50. Found {num_vertices} vertices."}

        
    # Ensure the query string is URL-encoded
    query_string = ensure_url_encoded(query_string)
    
    # Initialize base API URL
    api_url = f"https://api.tomtom.com/search/{versionNumber}/geometrySearch/{query_string}.{ext}?key={api_key}&limit={limit}"
        
    # Add additional parameters if they are not None
    optional_params = {
        #'limit': limit,
        'language': language,
        'extendedPostalCodesFor': extendedPostalCodesFor,
        'idxSet': idxSet,
        'categorySet': categorySet,
        'brandSet': brandSet,
        'connectorSet': connectorSet,
        'minPowerKW': minPowerKW,
        'maxPowerKW': maxPowerKW,
        'fuelSet': ','.join(fuelSet) if fuelSet else None,
        'view': view,
        'openingHours': openingHours,
        'timeZone': timeZone,
        'mapcodes': mapcodes,
        'relatedPois': relatedPois,
        'entityTypeSet': entityTypeSet
    }
        
    for key, value in optional_params.items():
        if value is not None:
            api_url += f"&{key}={value}"
    print(api_url)

    try:
        # Make the API request
        response = requests.post(api_url, json=polygon)
        
        # Parse and use the response
        if response.status_code == 200:
            pois = json.loads(response.text)
            logging.info(f"Successfully retrieved POIs. {len(pois)} items received.")
            return pois
        else:
            error_response = response.json()  # Get the JSON content of the response
            logging.warning(f"Failed to get data. Status code: {response.status_code}, Error: {error_response}")
            return {"error": f"Failed to get data: {response.status_code}, {error_response}"}
        
    except requests.RequestException as e:
        logging.error(f"Request failed: {e}. URL: {api_url}")
        return {"error": f"Request failed: {e}"}
    except json.JSONDecodeError as e:
        logging.error(f"Failed to decode JSON: {e}")
        return {"error": f"Failed to decode JSON: {e}"}
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return {"error": f"An unexpected error occurred: {e}"}


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

TOMTOM_API_KEY = os.environ.get("TOMTOM_API_KEY")
query_string = "Electric Vehicle Charging Station"

# You can now call the function with only the required arguments, or include optional ones.
result = get_pois_by_polygon(
    TOMTOM_API_KEY, payload, query_string
)
print(result)